# Making An Air Pollutant Forecasting model - Part I

<br>
Before we make our own air pollutant forecasting model, let's first try to describe the relationship between individual air pollutants and population (or, predict the values of those counties without measurements). 
The data science process you learn here will be very useful for building a useful forecasting tool later!

We will start by:
- using yearly average AQI
- using only one "feature" (population) for prediction
- using simple linear models: a straight line and a quadratic function

Let's start by loading in some libaries using the __`import`__ function

In [1]:
import numpy as np
import pandas as pd
import importlib
from IPython.display import Image
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from bokeh.io import output_notebook
from bokeh.layouts import gridplot, row
from bokeh.plotting import figure, show
output_notebook()

Loading BokehJS ...

Next, we load in our data table, and create two additional attributes: Logarithm of population and its squared value. The table below should look identical to the one we see in CODAP.

In [2]:
df=pd.read_csv('popair_2017.csv')
df["Log_pop"]=pd.Series(np.log10(df["Population"]))
df["Log_pop_sq"]=pd.Series(df["Log_pop"]**2)
df

,Unnamed: 0,State_name,County_name,Year,avgAQI_o3,avgAQI_co,avgAQI_no2,avgAQI_so2,avgAQI_pm25,Population,Log_pop,Log_pop_sq
0,0,Alabama,Baldwin,2017,37.536170,NaN,NaN,NaN,30.218182,212628,5.327620,28.383540
1,1,Alabama,Clay,2017,NaN,NaN,NaN,NaN,32.457627,13367,4.126034,17.024156
2,2,Alabama,Colbert,2017,35.808943,NaN,NaN,NaN,30.487395,54500,4.736397,22.433452
3,3,Alabama,DeKalb,2017,36.844011,NaN,NaN,NaN,32.886957,71617,4.855016,23.571182
4,4,Alabama,Elmore,2017,34.013043,NaN,NaN,NaN,NaN,81677,4.912100,24.128724
5,5,Alabama,Etowah,2017,38.208163,NaN,NaN,NaN,35.761062,102755,5.011803,25.118169
6,6,Alabama,Houston,2017,32.653061,NaN,NaN,NaN,34.838983,104346,5.018476,25.185099
7,7,Alabama,Jefferson,2017,36.646442,5.173611,20.301329,13.586777,38.120253,659197,5.819015,33.860938
8,8,Alabama,Madison,2017,39.495935,NaN,NaN,NaN,30.805085,361046,5.557563,30.886501
9,9,Alabama,Mobile,2017,36.558887,NaN,NaN,1.652047,32.558559,413955,5.616953,31.550162


Here, we can select an attribute you want to predict.

Type in __avgAQI_o3__, __avgAQI_co__, __avgAQI_no2__, __avgAQI_so2__, or __avgAQI_pm25__ in the double quote below.

In [3]:
attr = "avgAQI_no2" 

In [4]:
#These couple lines are just selecting a subset of table with that attribute availalbe.
X=df.loc[:,["Log_pop","Log_pop_sq"]][df[attr]>0]
y=df.loc[:,attr][df[attr]>0]

We will split the data into a training set and a hold out set according to our analysis plan.
![analysisplan](analysis.png)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2)
print("Number of counties have measurements for the selected pollutant = ",len(y))
print("Number of counties in the training data set = ",len(y_train))
print("Number of counties in the hold out (testing) data set = ",len(y_test))

Number of counties have measurements for the selected pollutant =  242
Number of counties in the training data set =  193
Number of counties in the hold out (testing) data set =  49


### Model 1: A Straight Line $y=a+bx$

In [6]:
import lm1_interactive
importlib.reload(lm1_interactive)
lm1_interactive.lm1_interactive(X_train.iloc[:,0],y_train)

Loading BokehJS ...

Loading BokehJS ...

You can alculate the Mean-Squared-Error (__MSE__) based on the model. Try to run a few different combinations of a and b. Comparing the plots above and MSEs. What do you notice?

In [10]:
a = -24.26
b = 7.2

MSE = sum((a+b*X_train.iloc[:,0]-y_train)**2)/len(y_train)
print("MSE = "+np.str(np.around(MSE,2)))

MSE = 28.52


### Model 2: A Quadratic Function $y=a+bx+cx^2$

In [ ]:
import lm2_interactive
importlib.reload(lm2_interactive)
lm2_interactive.lm2_interactive(X_train.iloc[:,0],y_train)

Again, you can alculate the Mean-Squared-Error (__MSE__) based on the model.

In [11]:
a = -3.72
b = -1.05
c = 0.81

MSE = sum((a+b*X_train.iloc[:,0]+c*X_train.iloc[:,0]**2-y_train)**2)/len(y_train)
print("MSE = "+np.str(np.around(MSE,2)))

MSE = 28.21


### Validate Model 1 and Model 2 using the hold out data set

We'll now validate the two models you explored earlier using the data set we intentionally set aside.

In [7]:
lr1 = LinearRegression().fit(np.array(X_train.iloc[:,0]).reshape(-1,1), y_train)
print("The best-fit line has an intercept (a) of "+np.str(np.around(lr1.intercept_,2))+
      " and a slope (b) of "+np.str(np.around(lr1.coef_[0],2)))

The best-fit line has an intercept (a) of -24.26 and a slope (b) of 7.2


In [8]:
lr2 = LinearRegression().fit(X_train, y_train)
print("The best-fit quadratic function has a, b, c "+ np.str(np.around(lr2.intercept_,2)), 
      np.str(np.around(lr2.coef_[0],2))+" and "+np.str(np.around(lr2.coef_[1],2)))

The best-fit quadratic function has a, b, c -3.72 -1.05 and 0.81


In [9]:
import lm_plot
importlib.reload(lm_plot)
lm_plot.lm_plot(X_train.iloc[:,0],y_train,X_test.iloc[:,0],y_test,lr1)
lm_plot.lm_plot(X_train.iloc[:,0],y_train,X_test.iloc[:,0],y_test,lr2)

Loading BokehJS ...

Loading BokehJS ...